In [1]:
%matplotlib inline
%load_ext rpy2.ipython
from ml import query
import math
from pbto2 import data

In [2]:
# x = pd.DataFrame({'tsi_min': [10, 20, 50, 60, 70, 90, 100], 'pbto2': [1, 2, 5, 6, 7, 9, 10]})
# x = x.set_index('tsi_min')
# idx = np.arange(x.index.min(), x.index.max() + 1, 10)
# print(idx)
# x = x.reindex(pd.Int64Index(idx, name='tsi_min')).interpolate()
# x.reset_index()

In [3]:
d = data.get_raw_data()
d = d[~d['gos-3'].isnull()]
d = d[~d['marshall'].isnull()]
#d = d[d['tsi_min'] <= 4320]
#d = d[d['tsi_min'] <= 1440]
d = d[d['tsi_min'] <= 2880]
d = d[d['pbto2'] > 0]
#d = d.iloc[np.random.permutation(np.arange(len(d)))[:5000],:]
d['gos-3-binary'] = d['gos-3'].apply(lambda x: 0 if x <= 3 else 1)
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8179 entries, 0 to 29515
Data columns (total 36 columns):
datetime                 8179 non-null datetime64[ns]
base_deficit_arterial    2134 non-null float64
base_excess_arterial     972 non-null float64
blood_gases              3082 non-null datetime64[ns]
hco3a                    3056 non-null float64
icp1                     4983 non-null float64
icp2                     2698 non-null float64
map                      4906 non-null float64
paco2                    3064 non-null float64
pao2                     3063 non-null float64
pbto2                    8179 non-null float64
pha                      3066 non-null float64
pupil_response_l         2844 non-null float64
pupil_response_r         2817 non-null float64
pupil_size_l             2219 non-null float64
pupil_size_r             2219 non-null float64
sao2                     2083 non-null float64
sheet_name               8179 non-null float64
spo2                     7934 non

In [4]:
outcome = 'gos-3'
feats = [
    ([outcome, 'uid', 'age', 'sex', 'gcs', 'marshall'],),
    (['pbto2'], ['ct_bin.*']),
    (['pao2', 'icp'], ['mean', 'p10', 'p990', 'std'])
]

d_feat = data.get_raw_feature_data(d)
d_in, n_frac = data.get_prepared_features(d_feat, feats, outcome=None)

print('Feature NA fractions:')
print(n_frac)
print('---------------------')

d_in.info()

Feature NA fractions:
feature
pao2_p990           0.015564
pao2_p10            0.015564
pao2_mean           0.015564
pao2_std            0.023346
gos-3-favalive      0.260700
__n_records__     257.000000
dtype: float64
---------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 257 entries, 461 to 1056
Data columns (total 19 columns):
age                257 non-null float64
gcs                257 non-null float64
gos-3              257 non-null float64
marshall           257 non-null float64
pao2_mean          257 non-null float64
pao2_p10           257 non-null float64
pao2_p990          257 non-null float64
pao2_std           257 non-null float64
pbto2_ct_bin_01    257 non-null float64
pbto2_ct_bin_05    257 non-null float64
pbto2_ct_bin_10    257 non-null float64
pbto2_ct_bin_15    257 non-null float64
pbto2_ct_bin_20    257 non-null float64
pbto2_ct_bin_25    257 non-null float64
pbto2_ct_bin_30    257 non-null float64
sex                257 non-null float64
gos-3-faval

In [5]:
d_in.to_csv('/Users/eczech/data/ptbo2/export/data_wide_bin_48hr.csv')